In [38]:
import json
import re
nombre = 'Vidal'
filename = 'd:/Facultad/Tesis/Twitter/201910-'+nombre+'.txt'

def take_url(json_data):
    try:
        url = json_data['urls'][0]['expanded_url']
    except:
        url = ''
    return url

def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    

def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def salvando_tweets(filename):
    dias = []
    frases = []
    urls = []
    verified = []
    with open(filename, 'r', encoding = "utf8") as fp:
        for i, line in enumerate(fp):
            # Para cada linea lee el json y extrae la fecha
            json_data = json.loads(line)
            url = take_url(json_data)
            tweet = json_data['text']
            url = json_data['urls']
            extra = json_data



            if tweet[0:2] != 'RT':
                user = json_data['user']
                frases.append(tweet)
                dias.append(json_data['created_at'])

                try:
                    verified.append(user['verified'])
                except:
                    verified.append(False)

                try:
                    urls.append(url[0]['url'])
                except:
                    urls.append(0)

        return frases, dias, urls, verified


In [39]:
import pandas as pd

def borrar_url(tweet, url):
    if urls != 0:
        try:
            t = tweet.replace(str(url), "")     
        except:
            t = tweet
    else:
        t = tweet
    return t

frases, dias, urls, verified = salvando_tweets(filename)


df = pd.DataFrame({'Tweets': frases, 'Fecha': dias, 'urls': urls, 'verified': verified})
df['Tweets_sin_url'] = df.apply(lambda row: borrar_url(row['Tweets'], row['urls']), axis = 1)
df.to_csv("Tweets_"+nombre+"_oct.csv")